<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/intro_cuantizacion/blob/master/lab_2_downcasting.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Laboratorio 2: Carga de Modelos de ML con Diferentes Tipos de Datos

En este laboratorio, cargarás modelos de ML en diferentes tipos de datos.

## Setup para Colab
Si estás corriendo este Notebook en Google Colab corre la celda de abajo, de lo contrario ignórala.

In [ ]:
!git clone https://github.com/wakusoftware/intro_cuantizacion.git

%cd intro_cuantizacion

!cp -r helper.py /content/

%cd /content/

!rm -rf intro_cuantizacion/

## Introducción a la Reducción de Precisión (Downcasting)

La reducción de precisión, también conocida como downcasting, es el proceso de convertir un dato de un tipo de precisión más alta a un tipo de precisión más baja. Este proceso puede ser útil en diversas aplicaciones, especialmente en el entrenamiento y despliegue de modelos de aprendizaje automático, debido a las siguientes razones:

- **Reducción del uso de memoria**: Al utilizar tipos de datos de menor precisión, se puede reducir significativamente la cantidad de memoria utilizada por un modelo.
- **Aumento del rendimiento**: Los cálculos en tipos de datos de menor precisión pueden ser más rápidos, lo que puede acelerar tanto el entrenamiento como la inferencia.
- **Cuantización**: La cuantización es una técnica común que utiliza la reducción de precisión para convertir los parámetros de los modelos de float32 a tipos como int8, bfloat16, etc.

Sin embargo, es importante tener en cuenta que la reducción de precisión también puede introducir errores de redondeo y reducir la exactitud del modelo. Por lo tanto, es esencial evaluar el impacto de la reducción de precisión en el rendimiento del modelo.

In [1]:
from helper import DummyModel

In [ ]:
model = DummyModel()

In [ ]:
model

### Inspección de Tipos de Datos

Antes de empezar con la conversión de modelos, vamos a crear una función para inspeccionar los tipos de datos de los parámetros en un modelo. Esto nos ayudará a entender cómo están almacenados los parámetros antes y después de la conversión.


In [ ]:
def print_param_dtype(model):
    for name, param in model.named_parameters():
        print(f"{name} está cargado en {param.dtype}")

In [ ]:
print_param_dtype(model)

## Conversión de Modelo: `float16`

- Convierte el modelo a una precisión diferente.

In [ ]:
# float 16
model_fp16 = DummyModel().half()

- Inspecciona los tipos de datos de los parámetros.

In [ ]:
print_param_dtype(model_fp16)

In [ ]:
model_fp16

- Ejecuta una inferencia simple usando el modelo.

In [ ]:
import torch

In [ ]:
dummy_input = torch.LongTensor([[1, 0], [0, 1]])

In [ ]:
# inferencia usando el modelo con float32
logits_fp32 = model(dummy_input)

In [ ]:
logits_fp32

In [ ]:
# inferencia usando el modelo con float16
try:
    logits_fp16 = model_fp16(dummy_input)
except Exception as error:
    print("\033[91m", type(error).__name__, ": ", error, "\033[0m")

## Conversión de Modelo: `bfloat16`

#### Nota sobre deepcopy
- `copy.deepcopy` hace una copia del modelo que es independiente del original. Las modificaciones que realices a la copia no afectarán al original, porque estás haciendo una "copia profunda". Para más detalles, consulta la documentación de Python sobre la biblioteca [copy](https://docs.python.org/3/library/copy.html).

In [ ]:
from copy import deepcopy

In [ ]:
model_bf16 = deepcopy(model)

In [ ]:
model_bf16 = model_bf16.to(torch.bfloat16)

In [ ]:
print_param_dtype(model_bf16)

In [ ]:
logits_bf16 = model_bf16(dummy_input)

- Ahora, compara la diferencia entre `logits_fp32` y `logits_bf16`.

In [ ]:
mean_diff = torch.abs(logits_bf16 - logits_fp32).mean().item()
max_diff = torch.abs(logits_bf16 - logits_fp32).max().item()

print(f"Mean diff: {mean_diff} | Max diff: {max_diff}")

## Uso de Modelos Generativos Populares en Diferentes Tipos de Datos

- Carga [Salesforce/blip-image-captioning-base](https://huggingface.co/Salesforce/blip-image-captioning-base) para realizar subtitulación de imágenes.

```Python
# Cargar el modelo directamente
from transformers import AutoProcessor, AutoModelForSeq2SeqLM

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/blip-image-captioning-base")
```

- Para ver el código de muestra con un ejemplo, haz clic en "Read model documentation" en la parte inferior del popup. Se abrirá una nueva pestaña.
  https://huggingface.co/docs/transformers/main/en/model_doc/blip#transformers.BlipForConditionalGeneration
- En esta página, desplázate un poco hacia abajo, más allá de la sección "parameters", y verás "Examples:"

```Python
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "A picture of"

inputs = processor(images=image, text=text, return_tensors="pt")

outputs = model(**inputs)
```

In [ ]:
from transformers import BlipForConditionalGeneration

In [ ]:
model_name = "Salesforce/blip-image-captioning-base"

In [ ]:
model = BlipForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# inspeccionar los tipos de datos predeterminados del modelo

# print_param_dtype(model)

- Verifica la huella (footprint) de memoria del modelo.

In [ ]:
fp32_mem_footprint = model.get_memory_footprint()

In [ ]:
print("Huella del modelo fp32 en bytes: ", 
      fp32_mem_footprint)
print("Huella del modelo fp32 en MBs: ", 
      fp32_mem_footprint/1e+6)


- Carga el mismo modelo en `bfloat16`.

In [ ]:
model_bf16 = BlipForConditionalGeneration.from_pretrained(
                                               model_name,
                               torch_dtype=torch.bfloat16
)

In [ ]:
bf16_mem_footprint = model_bf16.get_memory_footprint()

In [ ]:
# Obtener la diferencia relativa
diferencia_relativa = bf16_mem_footprint / fp32_mem_footprint

print("Huella del modelo bf16 en MBs: ", 
      bf16_mem_footprint/1e+6)
print(f"Diferencia relativa: {diferencia_relativa}")


### Rendimiento del Modelo: `float32` vs `bfloat16`

- Ahora, compara los resultados de generación de los dos modelos.

In [ ]:
from transformers import BlipProcessor

In [ ]:
processor = BlipProcessor.from_pretrained(model_name)

- Load the image.

In [ ]:
from helper import load_image, get_generation
from IPython.display import display

img_url = 'https://storage.googleapis.com/\
sfr-vision-language-research/BLIP/demo.jpg'

image = load_image(img_url)
display(image.resize((500, 350)))

In [ ]:
results_fp32 = get_generation(model, 
                              processor, 
                              image, 
                              torch.float32)

In [ ]:
print("Resultados del modelo con fp32:\n", results_fp32)

In [ ]:
results_bf16 = get_generation(model_bf16, 
                              processor, 
                              image, 
                              torch.bfloat16)

In [ ]:
print("Resultados del modelo con bf16:\n", results_bf16)

### Tipo de Dato Predeterminado

- Para la biblioteca Hugging Face Transformers, el tipo de dato predeterminado para cargar los modelos es `float32`.
- Puedes establecer el "tipo de dato predeterminado" como desees.

In [ ]:
desired_dtype = torch.bfloat16
torch.set_default_dtype(desired_dtype)

In [ ]:
dummy_model_bf16 = DummyModel()

In [ ]:
print_param_dtype(dummy_model_bf16)

- De manera similar, puedes restablecer el tipo de dato predeterminado a `float32`.

In [ ]:
torch.set_default_dtype(torch.float32)

In [ ]:
print_param_dtype(dummy_model_bf16)

### Nota
- Acabas de usar una forma simple de cuantización, en la cual los parámetros del modelo se guardan en un tipo de dato más compacto (bfloat16). Durante la inferencia, el modelo realiza sus cálculos en este tipo de dato, y sus activaciones están en este tipo de dato.
- En la próxima lección, usarás otro método de cuantización, "cuantización lineal", que permite que el modelo cuantizado mantenga un rendimiento mucho más cercano al modelo original al convertir del tipo de dato comprimido de vuelta al tipo de dato original FP32 durante la inferencia.